# 先寫一個簡單版的ptt爬蟲

In [15]:
import requests, json
from bs4 import BeautifulSoup

data = []
for page in range(25675, 25676):
    res = requests.get('https://www.ptt.cc/bbs/Gossiping/index.html', cookies={'over18': '1'})
    soup = BeautifulSoup(res.text)
    
    for j in soup.select('.title a'):
        res = requests.get('https://www.ptt.cc' + j['href'], cookies={'over18': '1'})
        soup = BeautifulSoup(res.text)
        data.append({'title':j.text, 'url':'https://www.ptt.cc' + j['href'], 'text':soup.select('#main-content')[0].text})

json.dump(data, open('ptt.json', 'w'))

c:\users\udic\documents\htdocs\publictutorial\venv\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file c:\users\udic\appdata\local\programs\python\python35\Lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "html5lib")

  markup_type=markup_type))


## 困難版

In [16]:
def parse(link, timeout=3):
    resp = requests.get(url=link, cookies={'over18': '1'}, verify=False, timeout=timeout)
    if resp.status_code != 200:
        return json.dumps({"error": "invalid url"}, sort_keys=True, ensure_ascii=False)
    soup = BeautifulSoup(resp.text, 'html.parser')
    main_content = soup.find(id="main-content")
    metas = main_content.select('div.article-metaline')
    title = ''
    if metas:
        title = metas[1].select('span.article-meta-value')[0].string if metas[1].select('span.article-meta-value')[0] else title

    # remove and keep push nodes
    pushes = main_content.find_all('div', class_='push')
    for push in pushes:
        push.extract()

    # 移除 '※ 發信站:' (starts with u'\u203b'), '◆ From:' (starts with u'\u25c6'), 空行及多餘空白
    # 保留英數字, 中文及中文標點, 網址, 部分特殊符號
    filtered = [ v for v in main_content.stripped_strings if v[0] not in [u'※', u'◆'] and v[:2] not in [u'--'] ]
    expr = re.compile(r'[^\u4e00-\u9fa5\u3002\uff1b\uff0c\uff1a\u201c\u201d\uff08\uff09\u3001\uff1f\u300a\u300b\s\w:/-_.?~%()]')
    for i in range(len(filtered)):
        filtered[i] = re.sub(expr, '', filtered[i])

    filtered = [_f for _f in filtered if _f]  # remove empty strings
    content = ' '.join(filtered)
    content = re.sub(r'(\s)+', ' ', content)

    # push messages
    p, b, n = 0, 0, 0
    messages = []
    for push in pushes:
        if not push.find('span', 'push-tag'):
            continue
        push_tag = push.find('span', 'push-tag').string.strip(' \t\n\r')
        push_userid = push.find('span', 'push-userid').string.strip(' \t\n\r')
        # if find is None: find().strings -> list -> ' '.join; else the current way
        push_content = push.find('span', 'push-content').strings
        push_content = ' '.join(push_content)[1:].strip(' \t\n\r')  # remove ':'
        push_ipdatetime = push.find('span', 'push-ipdatetime').string.strip(' \t\n\r')
        messages.append( {'push_tag': push_tag, 'push_userid': push_userid, 'push_content': push_content, 'push_ipdatetime': push_ipdatetime} )
        if push_tag == u'推':
            p += 1
        elif push_tag == u'噓':
            b += 1
        else:
            n += 1

    # json data
    data = {
        'article_title': title,
        'content': content,
        'messages': messages
    }
    return data

import requests, json, re
from bs4 import BeautifulSoup

data = []
for page in range(25675, 25676):
    res = requests.get('https://www.ptt.cc/bbs/Gossiping/index.html', cookies={'over18': '1'})
    soup = BeautifulSoup(res.text)
    
    for j in soup.select('.title a'):
        res = requests.get('https://www.ptt.cc' + j['href'], cookies={'over18': '1'})
        soup = BeautifulSoup(res.text)
        data.append(parse('https://www.ptt.cc' + j['href']))

json.dump(data, open('ptt.json', 'w'))

c:\users\udic\documents\htdocs\publictutorial\venv\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file c:\users\udic\appdata\local\programs\python\python35\Lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "html5lib")

  markup_type=markup_type))
c:\users\udic\documents\htdocs\publictutorial\venv\lib\site-packages\urllib3\connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarnin